In working through the SGCN problem with some new knowledge and thinking about interacting with ScienceBase, I'm experimenting here with a way to gather up every species name into one overall table to operate against. I still need to do a little thinking about how to deal with new data coming in over time in terms of how those should be stored in our final index. The process here is pretty straightforward and relies heavily on some Pandas fu. It uses sciencebasepy to grab every item from the source collection and then works them over to retrieve the flagged data files and take some initial cleanup steps.

This produces one overall dataset out of every state list surprisingly quickly. I think the idea will be to run this process periodically, using the file date from the ScienceBase-stored files to see if there are any new state lists that need to be updated. I just haven't figured out what all to store from older data and how to run the API to pull the most current information.

At the end, I dump this dataset to a CSV for later use. I experimented with the Pandas groupby capability which is really cool! Once we set up out clean_scientific_name field here, we can pull the CSV back into a dataframe, groupby that field, and have our unique names to lookup. After running the taxonomic authority consultation against ITIS and WoRMS, we can create an updated dataset with at least the ITIS TSN identifier for cases where it's best to go after related information with the ID vs. the name.

This new process should work in a much more efficient manner compared to the old. We should only be running and storing individual records from each of the sources we consult with (taxonomic authorities, related data systems) tied by name or ID to the SGCN list. We should really be able to cache all of this related information in a data store somewhere that is leveraged and continually updating based on any incoming vector. Species lists can consult with the cache to see what's there, call for an update (earlier than some type of logical schedule) if they want, and pull back whatever they want to work with.

In [1]:
from sciencebasepy import SbSession
from IPython.display import display
import pandas as pd
import requests
import json

import bispy

bis_utils = bispy.bis.Utils()
itis = bispy.itis.Itis()

In [2]:
sb = SbSession()

In [3]:
sgcn_base_item = sb.get_item('56d720ece4b015c306f442d5')

historic_national_list_file = next((f["url"] for f in sgcn_base_item["files"] if f["title"] == "Historic 2005 SWAP National List"), None)
if historic_national_list_file is not None:
    historic_national_list = requests.get(historic_national_list_file).text.split("\n")

In [4]:
def check_historic_list(scientificname):
    if scientificname in historic_national_list:
        return True
    else:
        return False

In [5]:
params = {
    "parentId": "56d720ece4b015c306f442d5",
    "fields": "title,dates,files,tags",
    "max": 500
}

items = sb.find_items(params)

sgcn_items = list()
while items and 'items' in items:
    sgcn_items.extend(items["items"])
    items = sb.next(items)

In [6]:
%%time
source_data = list()
for index, item in enumerate(sgcn_items):
    data_file = next(l["url"] for l in item["files"] if l["title"] == "Process File")
    
    try:
        df_src = pd.read_csv(data_file, delimiter="\t")
    except UnicodeDecodeError:
        df_src = pd.read_csv(data_file, delimiter="\t", encoding='latin1')
    
    # Make lower case columns to deal with slight variation in source files
    df_src.columns = map(str.lower, df_src.columns)

    # Set the file updated date from the ScienceBase file to each record in the dataset for future reference
    df_src["source_file_date"] = next(l["dateUploaded"] for l in item["files"] if l["title"] == "Process File")
    
    # Set the state name from the ScienceBase Item tag if needed
    if "state" not in df_src.columns:
        df_src["state"] = next(t["name"] for t in item["tags"] if t["type"] == "Place")

    # Set the reporting year from the ScienceBase Item date if needed
    if "year" not in df_src.columns:
        df_src["year"] = next(d["dateString"] for d in item["dates"] if d["type"] == "Collected")
    
    # Get rid of the reported '2005 SWAP' column because we can't count on it and it's too messy
    if "2005 swap" in df_src.columns:
        df_src.drop("2005 swap", axis=1, inplace=True)
        
    # Standardize naming of the reported taxonomic group column (though we may get rid of this eventually)
    if "taxonomy group" in df_src.columns:
        df_src.rename(columns={"taxonomy group": "taxonomic category"}, inplace=True)

    # Take care of the one weird corner case
    if "taxonomy group (use drop down box)" in df_src.columns:
        df_src.rename(columns={"taxonomy group (use drop down box)": "taxonomic category"}, inplace=True)

    # Clean up the scientific name string for lookup by applying the function from bis_utils
    df_src["clean_scientific_name"] = df_src.apply(lambda x: bis_utils.clean_scientific_name(x["scientific name"]), axis=1)

    # Check the historic list and flag any species names that should be considered part of the 2005 National List
    df_src["historic_list"] = df_src.apply(lambda x: check_historic_list(x["scientific name"]), axis=1)

    source_data.append(df_src)

# Put the individual dataframes together into one overall set
df_source = pd.concat(source_data, sort=True)

CPU times: user 11.1 s, sys: 1.08 s, total: 12.1 s
Wall time: 17.9 s


In [7]:
df_source.to_csv("sgcn_source_data.csv", index=False)